# Sensor fotônico
#### Bovine Brucellosis (Bactéria)

Análise das imagens de um sensor fotônico (guia de onda) para a detecção de uma bactéria através dos pontos de espalhamento gerados no processo de decantação

***
#### **Mestranda**
Bianca Tieppo
***
#### Orientação
Profa. Dra. Lúcia Akemi Miyazato Saito (UPM)
<br>Profa. Dra. Daniella Lopez Vale (UFRJ)
***

#### Criação do Dataframe através dos dados obtidos a partir das imagens

In [1]:
# Importar as bibliotecas padrões
%matplotlib inline 

import datetime
from math import sqrt
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from openpyxl import Workbook
import os
import seaborn as sn
import shutil

# Bibliotecas para processar imagens 
#1
import cv2

#2
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.io import imread, imshow
from skimage.measure import label, regionprops
from skimage.morphology import (erosion, 
                                dilation, 
                                closing, opening, 
                                area_closing, 
                                area_opening)

In [2]:
def current_path(): 
    print("Current working directory") 
    print(os.getcwd()) 
    print() 
  
current_path() 
os.chdir(r'C:\Users\Bruno\Processamento\BBD_Dataframe_vs7') 
current_path() 

Current working directory
C:\Users\Bruno\Processamento

Current working directory
C:\Users\Bruno\Processamento\BBD_Dataframe_vs7



In [3]:
DataCV = "DataCV.xlsx"
Directory = (r'C:\Users\Bruno\Processamento\BBD_Dataframe_vs7') 

DataCV_Path = os.path.join(Directory, DataCV)

# Verifica se o arquivo já existe
if not os.path.exists(DataCV_Path):
    # Cria um novo arquivo Excel
    workbook = Workbook()  
    # Salva o arquivo
    workbook.save(DataCV_Path)
    workbook.close()
    print(f"Arquivo {DataCV} criado com sucesso!")
else:
    print(f"Arquivo {DataCV} já existe no diretório {Directory}.")

Arquivo DataCV.xlsx já existe no diretório C:\Users\Bruno\Processamento\BBD_Dataframe_vs7.


In [4]:
def Image_positive(file, filename):
    width, height = file.shape[:2]
    
    remove_line = file[0:width, 400:700]
    # Aplicar rotação na matriz
    rotateddata = cv2.rotate(remove_line, cv2.ROTATE_90_COUNTERCLOCKWISE)
    # Converter a imagem para escala de cinza
    #grayimage = cv2.cvtColor(rotateddata, cv2.COLOR_BGR2GRAY)
    # Ajustar os parâmetros do filtro gaussiano para reduzir o espalhamento
    #filterimage = cv2.GaussianBlur(grayimage, (1, 1), 0)
    # Aplicar a binarização para identificação de linhas mais escuras
    _, binary_image = cv2.threshold(rotateddata, 60, 255, cv2.THRESH_BINARY)
    # Usar filtro canny para reconhecimento de bordas na matriz rotacionada
    imagecanny = cv2.Canny(binary_image, 10, 100) 

    firstpixel = np.where(imagecanny != 0)[0][0]
    lastpixel = np.where(imagecanny != 0)[0][-1]

    finaldata = rotateddata[firstpixel:lastpixel, 0:824]

    cv2.imwrite(filename, finaldata)
    
    return finaldata

In [5]:
def Image_negative(file, filename):
    width, height = file.shape[:2]
    
    remove_line = file[0:width, 400:700]
    # Aplicar rotação na matriz
    rotateddata = cv2.rotate(remove_line, cv2.ROTATE_90_COUNTERCLOCKWISE)
    # Converter a imagem para escala de cinza
    #grayimage = cv2.cvtColor(rotateddata, cv2.COLOR_BGR2GRAY)
    # Ajustar os parâmetros do filtro gaussiano para reduzir o espalhamento
    #filterimage = cv2.GaussianBlur(grayimage, (1, 1), 0)
    # Aplicar a binarização para identificação de linhas mais escuras
    _, binary_image = cv2.threshold(rotateddata, 60, 255, cv2.THRESH_BINARY)
    # Usar filtro canny para reconhecimento de bordas na matriz rotacionada
    imagecanny = cv2.Canny(binary_image, 50, 200) 

    firstpixel = np.where(imagecanny !=0)[0][0]
    lastpixel = np.where(imagecanny !=0)[0][-1]
    finaldata = rotateddata[firstpixel:lastpixel, 0:824]
    
    cv2.imwrite(filename, finaldata)
    
    return finaldata

In [6]:
def HSV_convert(data):
    # Obter valores na escala de cor de HSV 
    HSVfinaldata = cv2.cvtColor(finaldata, cv2.COLOR_BGR2HSV)
    # Obter matriz de valores de Hue
    H = HSVfinaldata[:,:,0]
    # Obter matriz de valores de Saturation
    S = HSVfinaldata[:,:,1]
    # Obter matriz de valores de Value
    V = HSVfinaldata[:,:,2]
    # Obter média das matrizes de H, S, V
    Hmean = np.mean(H)
    Smean = np.mean(S)
    Vmean = np.mean(V)

    #transformar valores em dataframe
    Hmean = pd.DataFrame(np.matrix(Hmean))
    Smean = pd.DataFrame(np.matrix(Smean))
    Vmean = pd.DataFrame(np.matrix(Vmean))
    
    return [Hmean, Smean, Vmean]

In [7]:
def Identify_spots(filename):
    # Lê a imagem usando o nome do arquivo fornecido
    img = cv2.imread(filename)
    
    # Converte a imagem para escala de cinza
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Aplica um desfoque gaussiano para suavizar a imagem
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)
    
    # Dilata a imagem suavizada usando uma matriz de kernel 5x5
    dilated_image = cv2.dilate(blurred, np.ones((5, 5)))
    
    # Aplica um limiar binário na imagem dilatada
    binary_threshold = 120
    _, thresh = cv2.threshold(dilated_image, binary_threshold, 255, cv2.THRESH_BINARY)

    # Criação de uma máscara vazia com o mesmo tamanho da imagem original
    mask = np.zeros(img.shape, dtype=np.uint8)

    # Encontrar contornos na imagem binarizada
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # Inicializa variáveis para armazenar o número total de pontos de espalhamento e a área total
    total_area = 0
    spots = len(cnts)
    
    # Percorre todos os contornos encontrados
    for c in cnts:
        area = cv2.contourArea(c)
        total_area += area

    # Cria uma máscara para desenhar os contornos (não utilizada no código atual)
    #smallest = sorted(cnts, key=cv2.contourArea)[0]
    #largest = sorted(cnts, key=cv2.contourArea)[-1]
    #cv2.drawContours(mask, [largest], -1, (255,255,255), -1)
    #cv2.drawContours(mask, [smallest], -1, (255,255,255), -1)

     # Aplica a máscara na imagem original para destacar os pontos de espalhamento
    result = cv2.bitwise_and(img, img, mask=thresh)
    result[thresh == 0] = (255,255,255)
    
    # Salva a imagem resultante com os pontos de espalhamento em um novo arquivo
    filename = filename.translate(str.maketrans('', '', '_proc.jpeg')) + '_spots.jpeg'
    cv2.imwrite(filename, result)
    
    # Retorna o número de pontos de espalhamento e a área total
    return [spots, total_area]

In [8]:
def Scikit_image(filename):
    # Lê a imagem usando a função imread do scikit-image
    im1 = imread(filename)
    
    # Converte a imagem para escala de cinza usando a função rgb2gray do scikit-image
    im2 = rgb2gray(imread(filename))
    
     # Converte a imagem em escala de cinza para uma imagem binária usando um limiar de 0.4
    im2_bw = im2<0.4

    # Aplica operações de fechamento e abertura na imagem binária usando um tamanho de elemento estruturante de 5
    img_morph = area_closing(area_opening(im2_bw, 5), 5)

    # Detecta blobs (regiões brilhantes) na imagem usando o Laplacian of Gaussian (LoG)
    blobs_log = blob_log(img_morph, max_sigma = 30, num_sigma = 10, threshold = 0.15)
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2) # Computa os raios na terceira coluna

     # Detecta blobs na imagem usando o Difference of Gaussian (DoG)
    blobs_dog = blob_dog(img_morph, max_sigma = 20, threshold = 0.10)
    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2) # Computa os raios na terceira coluna

    # Detecta blobs na imagem usando o Determinant of Hessian (DoH)
    blobs_doh = blob_doh(img_morph, max_sigma = 20, threshold = 0.05)

    # Cria uma lista com os blobs encontrados por cada método (LoG, DoG, DoH)
    blobs_list = [blobs_log, blobs_dog, blobs_doh]
    
    # Define as cores e os nomes de arquivo para salvar as imagens resultantes
    colors = ['yellow', 'lime', 'blue']
    filenames = [filename.translate(str.maketrans('', '', '_proc.jpeg')) + '_LoG.jpeg',
                filename.translate(str.maketrans('', '', '_proc.jpeg')) + '_DoG.jpeg',
                filename.translate(str.maketrans('', '', '_proc.jpeg')) + '_DoH.jpeg'] 
    
    # Combina as listas blobs_list, colors e filenames usando a função zip
    sequence = zip(blobs_list, colors, filenames)

    # Itera sobre cada combinação de blobs, cor e nome de arquivo
    for blobs, color, filename in sequence:
        # Cria uma figura e um eixo para plotar os blobs sobre a imagem original
        fig, ax = plt.subplots(1, 1, figsize = (9.3, 9.3))
        ax.imshow(im1, interpolation = 'nearest')
        
        # Desenha um círculo para cada blob detectado na imagem
        for blob in blobs:
            y, x, r = blob
            c = plt.Circle((x, y), r, color=color, linewidth=2, fill = False)
            ax.add_patch(c)
        
        ax.axis('off')
        
         # Salva a figura com os blobs destacados no arquivo correspondente
        plt.savefig(filename, bbox_inches='tight', pad_inches = 0)
        plt.close(fig)

    # Obtém o número de blobs encontrados por cada método
    LoG = len(blobs_log)
    DoG = len(blobs_dog)
    DoH = len(blobs_doh)
    
    # Retorna uma lista com o número de blobs encontrados por cada método
    return [LoG, DoG, DoH]    

In [13]:
def File_creation(filename):
    c_time = os.path.getmtime(filename)
    
    #date_c = datetime.datetime.fromtimestamp(c_time).date()
    #time_c = datetime.datetime.fromtimestamp(c_time).time()
    date_c = datetime.datetime.fromtimestamp(c_time)
    
    return [date_c]

In [14]:
folders_os = os.listdir(os.getcwd())
chips = ['positive', 'negative']
folders = []

for folder_os in folders_os:
    for chip in chips:
        if chip in folder_os:
            folders.append(folder_os)

In [15]:
folders

['01-03-23_positive1',
 '02-03-23_positive2',
 '02-03-23_positive3',
 '03-07-23_negative1',
 '03-07-23_positive1',
 '03-07-23_positive2',
 '04-07-23_negative1',
 '04-07-23_negative2',
 '05-06-23_positive1',
 '06-06-23_negative1',
 '06-06-23_negative2',
 '13-03-23_negative1',
 '13-03-23_negative2',
 '14-03-23_negative3',
 '15-06-23_negative1',
 '15-06-23_positive2',
 '29-06-23_negative1',
 '29-06-23_positive1',
 '29-06-23_positive2',
 '30-06-23_positive1',
 '30-06-23_positive2']

In [16]:
for folder in folders:
    path_wg = folder
    listFiles = os.listdir(path_wg)
    holding = {}
    
    if chips[0] in folder:
        bacteria = 'Positive'
    elif chips[1] in folder:
        bacteria = 'Negative'
    else:
        bacteria = '0'
    
    for file in listFiles:        
        if file.endswith('.jpg'):
            file_path = f"{path_wg}/{file}"
            holding[file] = cv2.imread(file_path,1)
            Creationdata = File_creation(file_path)

            filename = file.translate(str.maketrans('', '', '.jpg')) + '_proc.jpeg'
            
            if chips[0] in folder:
                finaldata = Image_positive(holding[file], filename)
            elif chips[1] in folder:
                finaldata = Image_negative(holding[file], filename)
                    
            HSVdata = HSV_convert(finaldata)
            Hmean = HSVdata[0]
            Smean = HSVdata[1]
            Vmean = HSVdata[2]

            imagedata = Identify_spots(filename)
            spots = imagedata[0]
            total_area = imagedata[1]

            imagedata_skimage = Scikit_image(filename)
            LoG = imagedata_skimage[0]
            DoG = imagedata_skimage[1]
            DoH = imagedata_skimage[2]

            # Abrir o arquivo xls
            # O arquivo Data.xls tem que existir na pasta antes de rodar o código
            Old_samples = pd.read_excel("DataCV.xlsx")

            # Criar um dataframe com os dados da amostra
            finaldata = pd.DataFrame({'Chip' : path_wg,
                                      'Name_of_sample' : file, 
                                      'Datetime': Creationdata,
                                      'ValueH': Hmean[0], 
                                      'ValueS': Smean[0],
                                      'ValueV': Vmean[0], 
                                      'Total_area': total_area,
                                      'Number_of_spots_cv2': spots, 
                                      'Number_of_spots_LoG': LoG,
                                      'Number_of_spots_DoG': DoG,
                                      'Number_of_spots_DoH': DoH,
                                      'Target': bacteria
                                     })

            # Concatenar a planilha antiga com a planilha nova
            concat_data = pd.concat([Old_samples, finaldata])

            # Salvar em cima do arquivo original os novos dados concatenados
            concat_data.to_excel('DataCV.xlsx', index = False)
        
    lista_arquivos = os.listdir(os.getcwd())
    lista_arquivos

    origem = r'C:\Users\Bruno\Processamento\BBD_Dataframe_vs7'
    os.chdir(origem)
    files = os.listdir(origem)
    files_a = [f for f in files if f.endswith('.jpeg')]

    destino = r'C:\Users\Bruno\Processamento\BBD_Dataframe_vs7' + '\\' + 'Processadas' + '\\' + path_wg
    os.makedirs(destino, exist_ok=True)
    
    for arquivo in files_a:
        shutil.move(arquivo, destino)

In [17]:
os.chdir(r'C:\Users\Bruno\Processamento\BBD_Dataframe_vs7')

In [18]:
df = pd.read_excel("DataCV.xlsx")

In [19]:
df.head()

,Chip,Name_of_sample,Datetime,ValueH,ValueS,ValueV,Total_area,Number_of_spots_cv2,Number_of_spots_LoG,Number_of_spots_DoG,Number_of_spots_DoH,Target
0,01-03-23_positive1,100min.jpg,2023-03-01 15:30:50,133.225104,240.931406,143.409167,434.0,8,139,103,6,Positive
1,01-03-23_positive1,105min.jpg,2023-03-01 15:35:20,134.332344,238.821302,146.889948,765.5,11,197,146,8,Positive
2,01-03-23_positive1,10min.jpg,2023-03-01 14:00:14,135.249566,246.915191,77.204861,0.0,0,0,0,0,Positive
3,01-03-23_positive1,110min.jpg,2023-03-01 15:39:46,127.686903,244.463487,141.553602,221.5,6,92,56,1,Positive
4,01-03-23_positive1,120min.jpg,2023-03-01 15:49:44,135.096705,240.158403,144.349559,497.0,8,185,110,11,Positive


In [20]:
df.shape

(718, 12)